<a href="https://colab.research.google.com/github/bl4ckf0xk/ModelX_First_Order/blob/main/ModelX_Model_With_Model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q scikit-optimize catboost imbalanced-learn lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.6 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, StratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, recall_score, precision_score
import joblib

In [3]:
!pip install shap
import shap

In [4]:
DATA_PATH = '/content/drive/MyDrive/Dementia Prediction Dataset.csv'
RANDOM_STATE = 42
TARGET = 'DEMENTED'
SUBJECT_ID = 'NACCID'  # change if your identifier column has a different name

In [31]:
def load_data(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Data file not found at {path}. Upload it to Colab or set DATA_PATH correctly.")
    df = pd.read_csv(path)
    return df

In [32]:
NON_MEDICAL_WHITELIST = [
    # Subject demographics
    'BIRTHYR', 'BIRTHMO', 'SEX', 'HISPANIC', 'HISPOR', 'HISPORX',
    'RACE', 'RACEX', 'RACESEC', 'RACESECX', 'RACETER', 'RACETERX',
    'EDUC', 'MARISTAT', 'PRIMLANG', 'PRIMLANX', 'RESIDENC', 'HANDED', 'NACCLIVS', 'INDEPEND',
    # Co-participant
    'INBIRYR', 'INBIRMO', 'INSEX', 'INHISP', 'INHISPOR', 'INHISPOX', 'INRACE', 'INRACEX', 'INRASEC', 'INRASECX', 'INRATER', 'INRATERX', 'INEDUC', 'INRELTO', 'INRELTOX', 'NEWINF',
    # Lifestyle
    'TOBAC30', 'TOBAC100', 'SMOKYRS', 'PACKSPER', 'QUITSMOK', 'ALCOCCAS', 'ALCFREQ',
    # Visit metadata
    'NACCVNUM', 'NACCNVST', 'NACCAVST', 'NACCDAYS', 'NACCFDYS', 'PACKET', 'FORMVER', 'TELCOV', 'TELMOD',
    # Family history (non-genetic fields)
    'NACCFAM', 'NACCMOM', 'NACCDAD'
]

In [33]:
print('Loading data...')
df = load_data(DATA_PATH)
print('Rows:', len(df), 'Columns:', len(df.columns))

Loading data...
Rows: 195196 Columns: 1024


In [34]:
# Ensure target and subject id exist
if TARGET not in df.columns:
    raise KeyError(f"Target column '{TARGET}' not found in dataset.")
if SUBJECT_ID not in df.columns:
    raise KeyError(f"Subject ID column '{SUBJECT_ID}' not found in dataset.")

# Reduce to columns we care about (keep subject id and target)
available_features = [c for c in NON_MEDICAL_WHITELIST if c in df.columns]
print(f'Using {len(available_features)} non-medical features (of {len(NON_MEDICAL_WHITELIST)} whitelist)')

# Warn about missing whitelist columns
missing = set(NON_MEDICAL_WHITELIST) - set(available_features)
if missing:
    print('Warning: The following whitelist columns were not found in your CSV and will be skipped:')
    print(sorted(list(missing)))

keep_cols = [SUBJECT_ID, TARGET] + available_features
df = df[keep_cols].copy()

Using 55 non-medical features (of 55 whitelist)


Handle NACC special codes -> convert common missing codes to NaN

In [35]:
# Typical NACC codes: -4 = Not applicable, -1 or 9 = Unknown, 88/99 = other/missing depending on variable
NA_CODES = [-4, -1, 8, 9, 88, 95, 96, 97, 98, 99, 999]
for v in df.columns:
    if df[v].dtype.kind in 'biufc':
        df[v] = df[v].replace(NA_CODES, np.nan)
    else:
        # for object/string fields, keep as-is and handle missing later
        df[v] = df[v].replace([str(x) for x in NA_CODES], np.nan)

# Also treat target code '9' explicitly as NaN (unknown)
df[TARGET] = df[TARGET].replace(9, np.nan)

# Drop rows where target is missing
df = df[df[TARGET].notna()].copy()
print('After dropping unknown target, rows:', len(df))

After dropping unknown target, rows: 195196


Feature engineering

In [36]:
# AGE if present; compute age-at-visit if possible using BIRTHYR and an approximate VISITYR if available.
# If NACCAGE exists, prefer it. Otherwise derive AGE from BIRTHYR with caveat.
if 'BIRTHYR' in df.columns and 'NACCFDYS' in df.columns:
    df['AGE'] = (df['NACCFDYS'] / 365.25) + (2023 - df['BIRTHYR'])
elif 'NACCAGE' in df.columns:
    df['AGE'] = df['NACCAGE']
else:
    df['AGE'] = np.nan

df['AGE_BIN'] = pd.cut(df['AGE'], bins=[0, 60, 70, 80, 120], labels=['<60', '60-70', '70-80', '80+'])

# Pack-years and simple booleans
if 'PACKSPER' in df.columns and 'SMOKYRS' in df.columns:
    df['PACK_YEARS'] = df['PACKSPER'] * df['SMOKYRS']
else:
    df['PACK_YEARS'] = np.nan

if 'TOBAC30' in df.columns and 'TOBAC100' in df.columns:
    df['EVER_SMOKE'] = ((df['TOBAC30'] == 1) | (df['TOBAC100'] == 1)).astype(int)
else:
    df['EVER_SMOKE'] = pd.Series([pd.NA] * len(df))

# Heavy alcohol heuristic
if 'ALCFREQ' in df.columns:
    # interpret codes: (user should adjust according to their codebook)
    # treat high frequency codes (e.g., weekly/daily) as heavy
    df['HEAVY_ALCOHOL'] = df['ALCFREQ'].apply(lambda x: 1 if (pd.notna(x) and float(x) >= 5) else 0).astype('Int64')
else:
    df['HEAVY_ALCOHOL'] = pd.Series([pd.NA] * len(df))

# Years in study
if 'NACCDAYS' in df.columns:
    df['YEARS_IN_STUDY'] = df['NACCDAYS'] / 365.25
else:
    df['YEARS_IN_STUDY'] = np.nan

# Lives alone bool from NACCLIVS (if 1 = lives alone in your version)
if 'NACCLIVS' in df.columns:
    df['LIVES_ALONE'] = df['NACCLIVS'].apply(lambda x: 1 if x == 1 else 0).astype('Int64')
else:
    df['LIVES_ALONE'] = pd.Series([pd.NA] * len(df))

df['MARRIED'] = (df['MARISTAT'] == 1).astype('Int64')  # 1 = married

# Education level (simplify)
df['HIGH_EDUC'] = (df['EDUC'] >= 16).astype('Int64')  # college+

# Family history of dementia
df['FAM_DEMENTIA'] = ((df['NACCMOM'] == 1) | (df['NACCDAD'] == 1)).astype('Int64')

df['AGE_EDUC_INTERACT'] = df['AGE'] * df['EDUC']
df['AGE_SMOKE_INTERACT'] = df['AGE'] * df['EVER_SMOKE']

# Final feature list (keep engineered features)
engineered = ['AGE', 'PACK_YEARS', 'EVER_SMOKE', 'HEAVY_ALCOHOL', 'YEARS_IN_STUDY', 'LIVES_ALONE', 'MARRIED', 'HIGH_EDUC', 'FAM_DEMENTIA', 'AGE_BIN', 'AGE_EDUC_INTERACT', 'AGE_SMOKE_INTERACT']
for e in engineered:
    if e not in df.columns:
        df[e] = np.nan

Final feature set building (numeric vs categorical)

In [37]:
FINAL_FEATURES = [
    'AGE', 'SEX', 'EDUC', 'MARISTAT', 'RESIDENC', 'INDEPEND',
    'LIVES_ALONE', 'MARRIED', 'HIGH_EDUC', 'EVER_SMOKE', 'PACK_YEARS', 'ALCFREQ',
    'FAM_DEMENTIA', 'NACCMOM', 'NACCDAD', 'HISPANIC', 'RACE', 'PRIMLANG',
    'AGE_BIN', 'HEAVY_ALCOHOL', 'YEARS_IN_STUDY', 'AGE_EDUC_INTERACT', 'AGE_SMOKE_INTERACT'
]

In [38]:
# After feature engineering, do this:
existing_features = [col for col in FINAL_FEATURES if col in df.columns]
missing_features = set(FINAL_FEATURES) - set(existing_features)

print(f"Using {len(existing_features)} features (of {len(FINAL_FEATURES)})")
if missing_features:
    print("Missing features (will be skipped):", sorted(missing_features))

Using 23 features (of 23)


In [41]:
from sklearn.model_selection import GroupShuffleSplit

X = df[existing_features].copy()
y = df[TARGET].astype(int)

print(y.value_counts(normalize=True))  # Check imbalance, e.g., 70% not demented

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, test_idx = next(gss.split(X, y, groups=df[SUBJECT_ID]))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

DEMENTED
0    0.704963
1    0.295037
Name: proportion, dtype: float64


In [42]:
numeric_features = [c for c in existing_features if df[c].dtype.kind in 'biufc']
categorical_features = [c for c in existing_features if c not in numeric_features]

print(f"Numeric: {len(numeric_features)}, Categorical: {len(categorical_features)}")

Numeric: 22, Categorical: 1


Preprocessing pipeline

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

In [44]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import warnings; warnings.filterwarnings('ignore')
from catboost import CatBoostClassifier

models = {
    'XGBoost': XGBClassifier(random_state=RANDOM_STATE, eval_metric='logloss'),
    'LightGBM': LGBMClassifier(random_state=RANDOM_STATE),
    'RandomForest': RandomForestClassifier(random_state=RANDOM_STATE),
    'Logistic': LogisticRegression(max_iter=1000),
    'CatBoost': CatBoostClassifier(random_state=RANDOM_STATE, verbose=0)
}

param_grids = {
    'XGBoost': {
        'classifier__n_estimators'     : [200, 300, 500],
        'classifier__max_depth'        : [4, 6, 8],
        'classifier__learning_rate'    : [0.01, 0.05, 0.1],
        'classifier__subsample'        : [0.8, 1.0],
        'classifier__colsample_bytree' : [0.8, 1.0]
    },
    'LightGBM': {
        'classifier__n_estimators' : [200, 300],
        'classifier__max_depth'    : [4, 6],
        'classifier__learning_rate': [0.05, 0.1],
        'classifier__num_leaves'   : [31, 63]
    },
    'RandomForest': {
        'classifier__n_estimators'      : [100, 200, 300],
        'classifier__max_depth'         : [5, 10, 15],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__class_weight'      : ['balanced', None]
    },
    'Logistic': {
        'classifier__C': [0.1, 1.0, 10.0]
    },
    'CatBoost': {
        'classifier__iterations'    : [200, 500],
        'classifier__depth'         : [4, 6, 8],
        'classifier__learning_rate' : [0.01, 0.1]
    }
}

Model pipeline + hyperparameter search

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from skopt import BayesSearchCV
from sklearn.metrics import (roc_auc_score, f1_score,
                             classification_report, confusion_matrix,
                             make_scorer)

best_models = {}
results = {}

def make_pipeline(model):
    return ImbPipeline([
        ('prep',  preprocessor),
        ('smote', SMOTE(random_state=RANDOM_STATE)),
        ('classifier', model)
    ])

cv_random = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

for name, base_model in models.items():
    print(f"\n=== Tuning {name} ===")
    pipe = make_pipeline(base_model)

    # Bayesian search for the three tree-based heavy models
    if name in ('XGBoost', 'LightGBM', 'CatBoost'):
        search = BayesSearchCV(
            estimator=pipe,
            search_spaces=param_grids[name],
            n_iter=25,                     # more efficient than 100 random draws
            cv=cv_random,
            scoring='roc_auc',
            n_jobs=-1,
            random_state=RANDOM_STATE,
            verbose=0
        )
    else:
        # Random search is fine for the simpler models
        search = RandomizedSearchCV(
            estimator=pipe,
            param_distributions=param_grids[name],
            n_iter=20,
            cv=cv_random,
            scoring='roc_auc',
            n_jobs=-1,
            random_state=RANDOM_STATE,
            verbose=0
        )

    search.fit(X_train, y_train)
    best_models[name] = search.best_estimator_

    # ---- evaluation on hold-out test set ----
    proba = search.predict_proba(X_test)[:, 1]
    pred  = search.predict(X_test)

    auc = roc_auc_score(y_test, proba)
    f1  = f1_score(y_test, pred)

    results[name] = {'auc': auc, 'f1': f1}

    print(f"{name} → AUC: {auc:.4f} | F1: {f1:.4f}")
    print(classification_report(y_test, pred))
    print("Confusion matrix:\n", confusion_matrix(y_test, pred))


=== Tuning XGBoost ===


In [ ]:
best_name = max(results, key=lambda k: results[k]['auc'])
best_model = best_models[best_name]
print(f"Best model: {best_name} (AUC={results[best_name]['auc']:.4f})")

from sklearn.calibration import CalibratedClassifierCV

# Calibrate for better probabilities
calibrated = CalibratedClassifierCV(best_model, method='isotonic', cv=3)
calibrated.fit(X_train, y_train)

Best Model: LightGBM (AUC: 0.9246)


CalibratedClassifierCV(cv='prefit',
                       estimator=Pipeline(steps=[('prep',
                                                  ColumnTransformer(transformers=[('num',
                                                                                   Pipeline(steps=[('imputer',
                                                                                                    SimpleImputer(strategy='median')),
                                                                                                   ('scaler',
                                                                                                    StandardScaler())]),
                                                                                   ['AGE',
                                                                                    'EDUC',
                                                                                    'PACK_YEARS']),
                                                                                  ('cat',
                                                                                   Pipeline(steps=[('imputer',
                                                                                                    SimpleImputer(strategy='most_frequent')),
                                                                                                   ('onehot',
                                                                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                                                                   ['SEX',
                                                                                    'MARISTAT',
                                                                                    'RESIDENC',
                                                                                    'INDEPEND',
                                                                                    'LIVES_ALONE',
                                                                                    'MARRIED',
                                                                                    'HIGH_EDUC',
                                                                                    'EVER_SMOKE',
                                                                                    'ALCFREQ',
                                                                                    'FAM_DEMENTIA',
                                                                                    'NACCMOM',
                                                                                    'NACCDAD',
                                                                                    'HISPANIC',
                                                                                    'RACE',
                                                                                    'PRIMLANG'])])),
                                                 ('model',
                                                  LGBMClassifier(learning_rate=0.05,
                                                                 max_depth=4,
                                                                 n_estimators=300,
                                                                 random_state=42))]))

In [ ]:
# SHAP for interpretability
explainer = shap.Explainer(calibrated.predict, preprocessor.fit_transform(X_train))
shap_values = explainer(preprocessor.transform(X_test))
shap.summary_plot(shap_values, preprocessor.transform(X_test), feature_names=best_model.named_steps['prep'].get_feature_names_out())

In [ ]:
import joblib
joblib.dump(calibrated, "dementia_risk_model_final.pkl")
print("Final model saved!")

Final model saved!


In [ ]:
# Load model
model = joblib.load("dementia_risk_model_final.pkl")

def predict_dementia_risk(user_data):
    """
    Example user_data dict with all keys (provide defaults or require all):
    user_data = {
        'AGE': 78, 'SEX': 2, 'EDUC': 10, 'MARISTAT': 1, 'RESIDENC': 1, 'INDEPEND': 1,
        'LIVES_ALONE': 1, 'MARRIED': 0, 'HIGH_EDUC': 0, 'EVER_SMOKE': 1, 'PACK_YEARS': 20,
        'ALCFREQ': 3, 'FAM_DEMENTIA': 1, 'NACCMOM': 1, 'NACCDAD': 0, 'HISPANIC': 0,
        'RACE': 1, 'PRIMLANG': 1, 'AGE_BIN': '70-80', 'HEAVY_ALCOHOL': 0,
        'YEARS_IN_STUDY': 5, 'AGE_EDUC_INTERACT': 780, 'AGE_SMOKE_INTERACT': 78
    }
    """
    # Create DataFrame with all expected columns
    df_input = pd.DataFrame([user_data],columns=FINAL_FEATURES)

    prob = model.predict_proba(df_input)[0, 1] * 100
    label = "At risk" if prob > 50 else "Not at risk"
    advice = (
        "High risk: Consult a doctor immediately and consider lifestyle changes like quitting smoking."
        if prob > 70 else
        "Moderate risk: Monitor health, increase education/mental activities."
        if prob > 30 else
        "Low risk: Keep up healthy habits!"
    )

    return {"risk_percent": round(prob, 2), "label": label, "advice": advice}

# Example
user = {
    'AGE': 78, 'SEX': 2, 'EDUC': 10, 'MARISTAT': 1, 'RESIDENC': 1, 'INDEPEND': 1,
    'LIVES_ALONE': 1, 'MARRIED': 0, 'HIGH_EDUC': 0, 'EVER_SMOKE': 1, 'PACK_YEARS': 20,
    'ALCFREQ': 3, 'FAM_DEMENTIA': 1, 'NACCMOM': 1, 'NACCDAD': 0, 'HISPANIC': 0,
    'RACE': 1, 'PRIMLANG': 1, 'AGE_BIN': '70-80', 'HEAVY_ALCOHOL': 0,
    'YEARS_IN_STUDY': 5, 'AGE_EDUC_INTERACT': 780, 'AGE_SMOKE_INTERACT': 78
}
result = predict_dementia_risk(user)
print(result)

{'risk_percent': 9.2, 'label': 'Not at risk', 'advice': 'Keep up healthy habits!'}
